In [1]:
import glob 
import cv2
from gtts import gTTS
import IPython.display as ipd

In [2]:
sift = cv2.SIFT_create()

In [3]:
index_params = dict(algorithm=0, trees=5)
search_params = dict()
bf = cv2.FlannBasedMatcher(index_params, search_params)

In [4]:
dic={10:0,20:0,50:0,100:0,200:0}

In [5]:
def sizeing(image,x,y):
    return  cv2.resize(image,(x, y))

In [6]:
def image_gray(image):
    Imagecolor=cv2.imread(image)#read image path ,flag=0:grayscale 1=color -1=alpha 
    Imagegray = cv2.cvtColor(Imagecolor, cv2.COLOR_BGR2GRAY)
    return Imagegray

take dictionary of grayscale image and calculate descriptor for each of them and return all descriptor as dictionary

In [7]:
def desc(des_of_images):
    descriptor={}
    for key, value in des_of_images.items():
        kp,des=sift.detectAndCompute(value,None)
        descriptor[key]=des
    return descriptor

take descriptor of new image for test and dictionary of  descriptor and calculate knnmatch and return all matches as dictionary

In [8]:
def knnMatch(des1,descriptor):
    one_verse_all={}
    for key,value in descriptor.items():
        one_verse_all[key]=bf.knnMatch(des1,descriptor[key],k=2)
    return one_verse_all

take  dictionary of matches and calculate best points to get and return all matches as dictionary

In [9]:
def valuess (one_verse_all):
    values={}
    for key, value in one_verse_all.items():
        good = []
        for m,n in value:
             if m.distance < .8*n.distance:
                good.append([m])
        values[key]=len(good)
    return values

take  dictionary of best points and  return index of max value in dictionary

In [10]:
def value(values):
    Keymax= max(values, key=values.get)
    return ((int)(Keymax.split(' ')[0]))

In [11]:
def total(dic):
    tot=0
    for i,j in dic.items():
        if (i*j!=0):
            tot+=i*j
    return tot

In [12]:
def cout(dic):
    string='In pocket,'
    for i,j in dic.items():
        string+=' '+str(j)+' of '+str(i)+','
    string+=" and the total is "+str(total(dic))
    return string

In [13]:
def texttoaudio(dic):
    language = 'en'
    myobj = gTTS(text=dic, lang=language, slow=False)   
    myobj.save("message.mp3")
    return "message.mp3"

In [14]:
def playAudio(txt):
    return ipd.Audio(txt, autoplay=True)

In [15]:
def func(image,des_of_images):
    Imagecolor=sizeing(image_gray(image),1200,500)
    kp1, des1 = sift.detectAndCompute(Imagecolor,None)
    descriptor=desc(des_of_images)
    one_verse_all=knnMatch(des1,descriptor)
    values=valuess(one_verse_all)
    dic[value(values)]=dic[value(values)]+1

In [16]:
des_of_images={}
src={}
i=0
for img in glob.glob("images_training/*.jpg"):
    Imagegray =image_gray(img)
    Imagegray = sizeing(Imagegray,1200,500)
    img=(img.split("\\")[1]).split(".")[0]
    des_of_images[img]=Imagegray
    src[i]=img
    i=i+1
print(src)

{0: '10 back', 1: '10 front', 2: '100 back', 3: '100 front', 4: '20 back', 5: '20 front', 6: '200 back', 7: '200 front', 8: '50 back', 9: '50 front'}


In [17]:
func("images_testing/200 back.jpg",des_of_images)

In [18]:
func("images_testing/200 back.jpg",des_of_images)

In [19]:
string=cout(dic)
string2=texttoaudio(string)
playAudio(string2)